# Notes

If using perfect approximate georeferencing (exactgeo),
everything works perfectly,
even for large paddings (up to 2).


With the following settings we can get some pretty good results.

- Only referenced images.
- Exact georeferencing with small padding.

The following variations on training sample.

- Near perfect: 10 training samples per camera.
- Very good: 10 training samples for the nadir camera. 1 training sample per other camera.
- Good: 10 training samples for the nadir camera only.
- Okay: 5 training samples per camera.


With the following settings we can get some pretty good results.

- Only referenced images.
- Approximate georeferencing with 0.5 padding.
- 10 training samples per camera

The following variations on training sample.

- 1.0 padding


# Setup


## Imports


In [ ]:
import os

In [ ]:
import pandas as pd
import yaml

In [ ]:
from night_horizons import utils

## Settings


In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

# Prepare Data


## Load Data


In [ ]:
# Discover y pred data
data = {}
y_pred_fps = utils.discover_data(settings['data_dir'], ['csv'], 'y_pred')
data['y_pred'] = [pd.read_csv(_, index_col=0) for _ in y_pred_fps]

In [ ]:
# Get the other data
for var_name in ['y_train', 'y_test', 'X_train', 'X_test', 'y_pred']:
    var_fps = y_pred_fps.str.replace('y_pred', var_name)
    data[var_name] = [pd.read_csv(_, index_col=0) for _ in var_fps]

In [ ]:
# Now get settings
mosaic_settings_fps = y_pred_fps.str.replace('y_pred.csv', 'settings.yaml')
nb_settings_fps = y_pred_fps.str.replace('y_pred.csv', 'settings.yaml')

mosaic_settings = []
nb_settings = []
for i, settings_fp in enumerate(mosaic_settings_fps):
    with open(settings_fp, "r", encoding='UTF-8') as file:
        mosaic_settings_i = yaml.load(file, Loader=yaml.FullLoader)
    mosaic_settings.append(mosaic_settings_i)

    with open(nb_settings_fps.iloc[i], "r", encoding='UTF-8') as file:
        nb_settings_i = yaml.load(file, Loader=yaml.FullLoader)
    nb_settings.append(nb_settings_i)
    

In [ ]:
# Finally, the logs
log_fps = y_pred_fps.str.replace('y_pred.csv', 'log.csv')
logs = [pd.read_csv(_, index_col=0) for _ in log_fps]

## Extract Stats
